In [1389]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import string
from sklearn.model_selection import train_test_split

In [1390]:
path = "../data/raw/titanic/train.csv"
path2 = "../data/raw/titanic/test.csv"
df = pd.read_csv(path)
df_test = pd.read_csv(path2)

In [1391]:
def alt(df):
    median = round(df["Age"].median(), 2)
    df["Age"] = df["Age"].fillna(median)

    mode = df["Embarked"].mode()[0]
    df["Embarked"] = df["Embarked"].fillna(mode)

    np.random.seed(499)
    null_id = df["Cabin"].index[df["Cabin"].isnull()]
    non_null = df["Cabin"].dropna().unique()
    df.loc[null_id, "Cabin"] = np.random.choice(non_null, size=len(null_id))

    df["Sex"] = df["Sex"].apply(lambda x: 1 if x == "male" else 0 if x == "female" else x)
    df["Embarked"] = df["Embarked"].apply(lambda x: 0 if x == "S" else 1 if x == "C" else 2 if x == "Q" else x)
    df["Cabin"] = df["Cabin"].apply(lambda x: ord(x[0]) if isinstance(x, str) else x)
    df["Fare"] = df["Fare"].apply(lambda x: 0 if len(str(x).split(".")[0]) == 2 else 1)
    df["Ticket"] = df["Ticket"].apply(lambda x: 0 if x[0].isalpha() else 1 if len(x) == 6 else 2 if len(x) == 4 else 3 if len(x) == 7 else 4 if len(x) == 5 else 5 if len(x) == 3 else x)
    df["Name"] = df["Name"].apply(lambda x: 0 if "Mrs" in x else 1 if "Mr" in x else 2 if "Miss" in x else 4)
    df["Age"] = df["Age"].apply(lambda x: 0 if x < 24 else 1)
    
    return df

df = alt(df)
df_test = alt(df_test)

In [1392]:
Y = df["Survived"]
X = df[["Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]]

In [1397]:
clf = KNeighborsClassifier()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state=3)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.776536312849162

In [1394]:
df_test["Survived"] = clf.predict(df_test.drop("PassengerId", axis=1))

In [1395]:
df_final = df_test[["PassengerId", "Survived"]]
df_final.to_csv("../data/processed/Final.csv", index=False)